In [1]:
import pandas as pd
import numpy as np
import csv
import re
import string
from itertools import product

In [2]:
test_file_list_A = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\listA.csv"
test_file_repeat_list_A = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\repeat read_list A.csv"
test_file_list_B = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\listB.csv"
test_file_repeat_list_B_end = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\repeat read_list B_end.csv"
test_file_repeat_listB_96 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\repeat read_listB_96.csv"
test_file = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Protein titration LissamineFA_22267.csv"
test_file_list_C = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\listC.csv"

In [8]:
reader = csv.reader(open(test_file_repeat_list_A, 'r'), delimiter=',')   
    
all_data = [] 

for row in reader:
    all_data.append(row)   

total_lines = len(all_data)

skiprows = total_lines + 1

for index, item in enumerate(all_data):
    if item != [] and len(item) == 1 and re.findall(r"Plate information", item[0]) == ["Plate information"]:
        skiprows_meta = index + 1

    if item != [] and len(item) >= 2 and re.findall(r"PlateNumber", item[0]) == ['PlateNumber'] and re.findall(r"PlateRepeat", item[1]) == ['PlateRepeat']:   # find line number with the beggining of the data
        skiprows = index - 1
        data_format = 'A'

    if item != [] and len(item) >= 2 and re.findall(r"Plate", item[0]) == ['Plate'] and re.findall(r"Barcode", item[1]) == ['Barcode']:   # find line number with the beggining of the data
        skiprows = index
        data_format = 'B'

    if item != [] and len(item) >= 2 and re.findall(r"Plate", item[0]) == ['Plate']  and re.findall(r"Well", item[1]) == ['Well']:
        skiprows = index
        data_format = 'C'

    if item != [] and re.findall(r"G-factor", item[0]) == ["G-factor"]:
        g_factor = float(item[4])
            
    if item == [] and index > skiprows:   # find the index of the first blank row after the data table
        end_of_data = index
        break  

nrows = end_of_data - skiprows - 1
nrows_meta = (skiprows_meta + int(all_data[skiprows_meta + 1][1]) + 3) - skiprows_meta

data = pd.read_csv(test_file_repeat_list_A, sep=',', engine='python', skiprows=skiprows, nrows=nrows, encoding='utf-8')
metadata = pd.read_csv(test_file_repeat_list_A, sep=',', engine='python', skiprows=skiprows_meta, nrows=nrows_meta, encoding='utf-8')
    
print(skiprows)
print(end_of_data)
print(nrows)
print(skiprows_meta)
print(nrows_meta)

#if data_format == 'A':
    #skipfooter = total_lines - end_of_data     
# lists B and C have data table headers composed of two lines
#if data_format == 'B' or data_format == 'C':
    #skipfooter = total_lines - end_of_data - 1 

193
1059
865
184
4


In [9]:
g_factor

1.15

In [10]:
data_format

'A'

In [11]:
metadata

,Plate,Repeat,Barcode,Measured height,Chamber temperature at start,Chamber temperature at end,Humidity at start,Humidity at end,Ambient temperature at start,Ambient temperature at end,Measurement date,Unnamed: 11
0,1,1,NaN,14.4,18.98,18.8,61.7,61.5,18.98,18.9,17/11/2020 13:33:43,NaN
1,1,2,NaN,14.4,18.80,18.7,61.5,61.8,18.90,19.1,17/11/2020 13:35:26,NaN


In [12]:
data

,PlateNumber,PlateRepeat,Barcode,Group,Well,Label,Kinetic,ScanX,ScanY,MeasInfo,MeasTime,Signal,Flashes/Time,Result,Formula,Unnamed: 15
0,1,1,NaN,1.0,A01,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:00:10.511,18781491.0,60.0,18781491.0,NaN,NaN
1,1,1,NaN,1.0,A01,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:00:10.186,20469296.0,60.0,20469296.0,NaN,NaN
2,1,2,NaN,1.0,A01,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:01:54.400,18964719.0,60.0,18964719.0,NaN,NaN
3,1,2,NaN,1.0,A01,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:01:54.075,20296635.0,60.0,20296635.0,NaN,NaN
4,1,1,NaN,NaN,A01,NaN,0,0,0,0,NaN,NaN,NaN,-26.8,Calc 1: mP value for FP measurement = 1000 * (...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,1,1,NaN,1.0,K24,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:01:52.623,13945420.0,60.0,13945420.0,NaN,NaN
860,1,2,NaN,1.0,K24,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:03:36.187,13026020.0,60.0,13026020.0,NaN,NaN
861,1,2,NaN,1.0,K24,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:03:36.512,13838929.0,60.0,13838929.0,NaN,NaN
862,1,1,NaN,NaN,K24,NaN,0,0,0,0,NaN,NaN,NaN,-38.0,Calc 1: mP value for FP measurement = 1000 * (...,NaN
